# New contrast method

Testing out a new contrast method. 

In [ ]:
from hydra import initialize, compose
from omegaconf import OmegaConf

# Initialize Hydra with the directory where your config lives.
# Note that hydra will tkae care of composing all our disparate config files
with initialize(config_path="conf", job_name="notebook_app"):
    # Compose the configuration, using "train" as the config name.
    cfg = compose(config_name="train", 
                  overrides=[
                      "model=vanilla_contrast_v3",
                      "scene.data_root=/home/ubuntu/cs-747-project/adt_processed",
                      "scene.scene_name=Apartment_release_work_skeleton_seq131",
                      "output_root=/home/ubuntu/cs-747-project/output/adt",
                      "exp_name=3dgs_new_contrast-1",
                      "lift.use_contr=True",
                      "wandb.project=egolifter_adt"
                  ])

In [ ]:
# Now you can use cfg to see what was loaded.
print(OmegaConf.to_yaml(cfg))

In [ ]:
# Make the output directory
import os
os.makedirs(cfg.scene.model_path, exist_ok=True)

In [ ]:
# Set up the logger (wandb)
from lightning.pytorch.loggers import WandbLogger

# Make the wandb directory
os.makedirs(os.path.join(cfg.scene.model_path, "wandb"), exist_ok=True)
os.makedirs(cfg.wandb.save_dir, exist_ok=True)

# Create the logger
logger = WandbLogger(
    project=cfg.wandb.project, 
    entity=cfg.wandb.entity,
    name=cfg.exp_name,
    save_dir=cfg.wandb.save_dir,
)

# Tell the logger what hyperparameters to log
logger.log_hyperparams(OmegaConf.to_container(cfg, resolve=True))

In [ ]:
# (OPTIONAL) And save the config to the output directory
# This is useful for keeping track of what you ran
OmegaConf.save(cfg, os.path.join(cfg.scene.model_path, "config.yaml"), resolve=True)

In [ ]:
import lightning as L
from scene import Scene

# Set the seed for reproducibility
L.seed_everything(cfg.seed)

# Create a new scene object
scene = Scene(cfg)

In [ ]:
from model import get_model

# Load the model. This is one of our LightningModules (i.e., VanillaGaussian, Unc2DUnet, etc.)
model = get_model(cfg, scene)
print(model)

In [ ]:
# This will load an initial point cloud. The point cloud is loaded from scene.scene_info.point_cloud, which was initialized
# above in the Scene class. Internally, that comes from global_points.csv.gz file (Aria dataset only; other datasets 
# init this differently).
model.init_or_load_gaussians(
    scene.scene_info.point_cloud,
    scene.scene_info.nerf_normalization["radius"], # NOTE: not sure that this does... 
    cfg.scene.model_path,
    load_iteration = None,
)

In [ ]:
# Loop over the model to print the parameters
for name, param in model.named_parameters():
    print(name, param.shape)

In [ ]:
# Load the data loader. This is a PyTorch DataLoader object that will load the data for training.
train_loader = scene.get_data_loader("train", shuffle=True, num_workers=cfg.scene.num_workers)
valid_loader = scene.get_data_loader("valid", shuffle=False, num_workers=cfg.scene.num_workers)
valid_novel_loader = scene.get_data_loader("valid_novel", shuffle=False, num_workers=cfg.scene.num_workers)

In [ ]:
from callback.checkpoint import ModelCheckpoint

# Set up the checkpoint callback. This will save the model every n steps.
checkpoint_callback = ModelCheckpoint(
    dirpath=cfg.scene.model_path,
    filename="chkpnt{step}",
    save_top_k=-1,
    verbose=True,
    monitor=None,
    every_n_train_steps = cfg.opt.ckpt_every_n_steps,
)

# Init the trainer
trainer = L.Trainer(
    max_steps=cfg.opt.iterations,
    logger=logger,
    check_val_every_n_epoch=None,
    val_check_interval = cfg.opt.val_every_n_steps,
    callbacks=[checkpoint_callback],
    devices=cfg.gpus, 
)

In [ ]:
# Train the model!
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=[valid_loader, valid_novel_loader],
)

# Eval

In [ ]:
import pandas as pd
# Evaluation
if not cfg.skip_test:
    for subset in ["test", "valid", "valid_novel", "train"]:
        loader = scene.get_data_loader(subset, shuffle=False)
        if len(loader) > 0:
            trainer.test(
                model=model,
                dataloaders=loader,
            )
            df = pd.DataFrame(model.test_logs)
            df.to_csv(os.path.join(cfg.scene.model_path, f"eval_logs_{subset}.csv"), index=False)

In [ ]:
from utils.eval_2dseg import eval_query_2dseg
import wandb
from pathlib import Path

# Evaluate the 2D segmentation
if cfg.lift.use_contr:
    if scene.scene_info.query_2dseg is None:
        print("No 2D segmentation query found in the scene info. Skipping 2D segmentation evaluation.")
    else:
        # Copy the to a new model to avoid a weird memory illegal access error
        model_eval = get_model(cfg, scene)
        model_eval.init_gaussians_size_from_state_dict(model.state_dict())
        model_eval.load_state_dict(model.state_dict())
        model_eval = model_eval.eval().cuda()
        
        for subset in ["test", "valid", "valid_novel", "train"]:
            print(f"Evaluating subset: {subset} ...")
            dataloader = scene.get_data_loader(subset, shuffle=False, num_workers=0, limit=200)
            threshold_mode = "fixed"
            threshold_value = 0.6
            static_miou, dynamic_miou, df_eval_logs = eval_query_2dseg(
                scene, dataloader, model_eval, threshold_mode, threshold_value)
            print(f"{subset}: static mIoU: {static_miou}, dynamic mIoU: {dynamic_miou}")
            
            wandb.log({
                f"2dseg_static/{subset}_miou": static_miou,
                f"2dseg_dynamic/{subset}_miou": dynamic_miou,
            })
            
            # Save the evaluation logs to the ckpt_folder
            save_path = Path(cfg.scene.model_path) / "2dseg_eval" / f"{subset}_logs_{threshold_mode}_{threshold_value}.csv"
            save_path.parent.mkdir(parents=True, exist_ok=True)
            df_eval_logs.to_csv(save_path, index=False)

In [ ]:
import os

os.environ["SCENE_NAME"] = cfg.scene.scene_name
os.environ["ADT_PROCESSED_ROOT"] = cfg.scene.data_root
os.environ["FOLDER_NAME"] = 'vanilla_3dgs'
os.environ["OUTPUT_ROOT"] = cfg.output_root

In [ ]:
# Render the images into a video 
# Select render_subset as one of the subset of images to render
# render_subset=novel        # novel subset
# render_subset=trainvalid   # seen subset

!uv run python render_lightning.py \
    model_path=$OUTPUT_ROOT/$SCENE_NAME/$FOLDER_NAME \
    render_subset=novel \
    source_path=$ADT_PROCESSED_ROOT/$SCENE_NAME